In [ ]:
# check if instruction only has 0,1, and new line chars
# check if instruction is exactly 8 bits
def check_inst(inst):
    # instruction is smaller than 8 bits, ignor it
    if len(inst)<8: 
        return False
    # truncate instruction to only 8 bits!
    inst = inst[:8]
    for bit in inst:
        if bit not in "01":
            return False
    return True

In [ ]:
# increment value of str binary
def incr_val(val): 
    val=int(val,2)+1
    if val > 15:
        # if overflow of the value (bigger than 15 or 2^4), return marker!
        return "X11X"
    else:
        #else, return 4 bits of the vale after incrementing
        return str(bin(val))[2:].zfill(4)

In [ ]:
def write_mem(val):
    # write to mem function. make sure that the value being written is
    # no longer than 8 bits. If it is, truncate!
    if len(val) > 8:
        val = val[:8]
    return val

In [ ]:
# ALU functions!
def ALU(op,v1="0000",v2="0001"):
    # if the ALU recieves a "000" op, it just Idels! nothing is is done!
    if op == "000":
        return "1XXX" # send special marker!
    
    # Otherwise, check if ADD instruction!
    elif op == "011":
        sum=int(v1,2)+int(v2,2) # add integer values into sum
        if sum > 15: # if bigger than (2^4)-1, return 0
            res = "0000"
        else:
            # else, result is binary of sum, then fill until result is
            # 4 bits long!
            res = str(bin(sum))[2:].zfill(4)
        return res
    # Otherwise, check if SUB instruction!
    elif op == "100": #subtract v2 from v1, return subtraction
        sub=int(v1,2)-int(v2,2)
        if sub < 0: # if smaller than 0, return 0
            res = "0000"
        else:
            # else, result is binary of subtraction, then fill until result 
            # is 4 bits long!
            res = str(bin(sub))[2:].zfill(4)
        return res
    # Otherwise, check if EQ instruction!
    elif op == "101":
        if v1 == v2 : #check if v1 equals v2
            # if yes, return true
            return True
        else:
            # if no, return False
            return False
    else:
        # If operation not implemented, return do not cares!!
        return "XXXX"

#instructions parser!

In [ ]:
def pars_8bit_instructions(instruction_file):
    # open instructions file!
    instruction_file = open(instruction_file)
    # read all lines and log them
    instructions = instruction_file.readlines()
    
    # codify symbols!
    inst_txt={
        "000":"IDL",
        "001":"LD ",
        "010":"ST ",
        "011":"ADD",
        "100":"SUB",
        "101":"EQ ",
        "110":"JP ",
        "111":"HE ",
        "0":"$",
        "1":"#"
    }
    
    # make a list of the instructions!
    inst_list=[]
    
    # go over each instruction and extract the op_code,
    # id, and operand from the instruction. Also create
    # the test representation of the the instruction
    for instruction in instructions:
        # check if instruction is ok
        if check_inst(instruction):
            # create dictionary of instruction values
            inst_dict={
                "op_code":instruction[:3],
                "id":instruction[3],
                "operand":instruction[4:8],
                # text representation of the instruction
                "text": inst_txt[instruction[:3]]+" "+ inst_txt[instruction[3]] +" "+instruction[4:8]
            }
            # add to instruction list
            inst_list.append(inst_dict)
    # once done reading instruction file, return instruction list!
    return inst_list

In [ ]:
def dummy_8(instruction_file,CYCLE_TIME=1):
    instruction_list = pars_8bit_instructions(instruction_file)
    mem = {
        "0000":"XXXXXXXX",
        "0001":"XXXXXXXX",
        "0010":"XXXXXXXX",
        "0011":"XXXXXXXX",
        "0100":"XXXXXXXX",
        "0101":"XXXXXXXX",
        "0110":"XXXXXXXX",
        "0111":"XXXXXXXX",
        "1000":"XXXXXXXX",
        "1001":"XXXXXXXX",
        "1010":"XXXXXXXX",
        "1011":"XXXXXXXX",
        "1100":"XXXXXXXX",
        "1101":"XXXXXXXX",
        "1110":"XXXXXXXX",
        "1111":"XXXXXXXX"
    }
    reg = {
        "PC":"0000", # Program Counter Value
        "IR":"XXX", # Instruction Counter Value
        "AC":"XXXX" # Accumulator value
    }
    cycles = 0
    mem_loc = "0000"
    
    #load instructions to mem
    for inst in instruction_list:
        print(inst) # print list of instructions!
        mem[mem_loc] = write_mem(inst["op_code"]+inst["op_code"]+inst["operand"])
        mem_loc = incr_val(mem_loc)
    # add header for the trace
    print("|------------------------------------------------------|")
    print("|-PC--|----IR-----|--AC--|----INST----|-CYCLES--|-TIME-|")
    # loop until either there is an overflow of the PC (No HE instruction is used)
    # or when the HE instruction is reached
    
    while (incr_val(reg["PC"])!="X11X"):
        # Halt execution if number of instruction bigger than PC
        # No HE instruction is used!
        if (int(reg["PC"],2) >= len(instruction_list)):
            print("****No HE instruction is used. Use HE to halt execution****")
            break
        # get the instrrction where the PC is pointing
        inst = instruction_list[int(reg["PC"],2)]
        
        # instruction decode
        op_code = inst["op_code"]
        operand_type = inst["id"]
        operand = inst["operand"]
        
        # load the instruction to the IR
        reg["IR"]=op_code+operand_type+operand
        
        instructionCycle=" "+reg["PC"]+" |"+" "*2+reg["IR"]+" |"+" "+reg["AC"] +" |"+" "+ inst['text']
        reg["PC"]=incr_val(reg["PC"])
        
        # Halt execution if inst is HE
        if (op_code=="111"): #HE
            cycles = cycles+1
            print(instructionCycle+" |  "+ str(cycles)+"\t| "+ str(round(cycles*CYCLE_TIME,2)))
            print("|------------------------------------------------------|")
            break
        
        # create idel situation!
        if reg["IR"][:3]=="000": # idel
            cycles = cycles+1
            
         # Load operand to the accumulator
        if reg["IR"][:3]=="001": # LD
            if operand_type=='1':
                reg["AC"]=operand
                cycles = cycles+1
            else:
                reg["AC"]=mem[operand]
                cycles = cycles+2
        
        # Store operand from the accumulator to mem address
        elif reg["IR"][:3]=="010": # ST
            mem[operand] = write_mem(reg["AC"])
            cycles = cycles+1
        
        # Add operand to value of accumulator
        elif reg["IR"][:3]=="011": #add
            if operand_type=='1':
                reg["AC"] = ALU(op_code,reg["AC"],operand)
                cycles = cycles+1
            else:
                reg["AC"] = ALU(op_code,reg["AC"],mem[operand])
                cycles = cycles+2
        
        # Subtract value of accumulator from operand 
        elif reg["IR"][:3]=="100": #sub
            if operand_type=='1':
                reg["AC"] = ALU(op_code,reg["AC"],operand)
                cycles = cycles+1
            else:
                reg["AC"] = ALU(op_code,reg["AC"],mem[operand])
                cycles = cycles+2
            
        # Check if operand of accumulator equals value
        elif reg["IR"][:3]=="101": #EQ
            if operand_type=='1':
                if ALU(op_code,reg["AC"],mem[operand]):
                    reg["PC"] = incr_val(reg["PC"])
            else:
                if ALU(op_code,reg["AC"],operand):
                    reg["PC"] = incr_val(reg["PC"])
            cycles = cycles+1
        
        # Jump to value of operand
        elif op_code=="110": #JP
            reg["PC"] = operand
            cycles = cycles+1
        time = round(cycles*CYCLE_TIME,2) 
        print(instructionCycle+" |  "+ str(cycles)+"\t| "+ str(time))
    else:
        print("PC value overflow --- ending infinate loop")
    return mem,reg,(cycles,time)



In [ ]:
%%time
dummy_8("instructions_example_dummy_8bit_CPU.txt",1.145)